In [1]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import csv
import xlrd
import os
import os.path
from pathlib2 import Path
from decimal import *
%matplotlib inline
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'font.weight':'medium', 
         'xtick.major.size':8,
         'ytick.major.size':8}
pylab.rcParams.update(params)

In [2]:
filedir = '/gpfs/amarolab/monolayers/'

In [5]:
mix1_1 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20-3_100ns/concat.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20-3_100ns/step9_1.gro')

In [4]:
mix1_2 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20_20ns/step9_1.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20_20ns/step9_1.gro')

In [6]:
mix1_3 = md.load(filedir+'MIX1_NaCl0.4_GMX/SA20-2_20ns/step9_1.trr', top=filedir+'MIX1_NaCl0.4_GMX/SA20_20ns/step9_1.gro')

In [9]:
filedir1 = '/gpfs/amarolab/abbyflabby/'
mix1_4 = md.load(filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20/charmm-gui/namd/step7.1_production.dcd', top=filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20/charmm-gui/step5_assembly.xplor_ext.psf')

In [13]:
dcds_mix1_5 = [filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.2_production.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.3_production.inp.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.31_production.inp.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.32_production.inp.dcd', filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-2/charmm-gui/namd/step7.33_production.dcd']
mix1_5 = md.load(dcds_mix1_5, top=filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20/charmm-gui/step5_assembly.xplor_ext.psf')     


In [14]:
dcds_mix1_6 = [filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.1_production.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.2_production.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.3_production.inp.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.31_production.inp.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.32_production.inp.dcd', 
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.33_production.dcd',
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.34_production.dcd',
               filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/namd/step7.35_production.dcd']
mix1_6 = md.load(dcds_mix1_5, top=filedir1+'MIX/MIX1234_NaCl0.4_NAMD/SA20-3/charmm-gui/step5_assembly.xplor_ext.psf') 

In [75]:
def get_nematic_orders(traj):
    traj.center_coordinates()
    residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']
    atoms = []
    for i in residues:
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]<0 and atom.element.symbol == 'C'])
    return(md.compute_nematic_order(traj, indices=atoms))
no6 = get_nematic_orders(mix1_6)
no7 = get_nematic_orders(mix1_5)
no8 = get_nematic_orders(mix1_4)
no9 = get_nematic_orders(mix1_3)
no10 = get_nematic_orders(mix1_2)
no11 = get_nematic_orders(mix1_1)

In [76]:
print(np.mean(no6))
print(np.mean(no7))
print(np.mean(no8))
print(np.mean(no9))
print(np.mean(no10))
print(np.mean(no11))

0.30196279277234306
0.3019627927825196
0.2995649011083628
0.2869519578460875
0.28689771834092515
0.2863010298226997


In [77]:
def get_nematic_orders_chains(traj):
    traj.center_coordinates()
    residues = [residue.index for residue in traj.top.residues if residue.name =='PALP' or residue.name == 'MYRP' or residue.name=='LAUP' or residue.name=='STEP']
    atoms = []
    for i in residues:
        atoms.append([atom.index for atom in traj.top.atoms if atom.residue.index == residues[i] and traj.xyz[0][atom.index][2]>0])
    return(md.compute_nematic_order(traj, indices=atoms))
no6C = get_nematic_orders_chains(mix1_6)
no7C = get_nematic_orders_chains(mix1_5)
no8C = get_nematic_orders_chains(mix1_4)
no9C = get_nematic_orders_chains(mix1_3)
no10C = get_nematic_orders_chains(mix1_2)
no11C = get_nematic_orders_chains(mix1_1)

In [78]:
print(np.mean(no6C))
print(np.mean(no7C))
print(np.mean(no8C))
print(np.mean(no9C))
print(np.mean(no10C))
print(np.mean(no11C))

0.3500819499403099
0.35008194994052383
0.3600483691026408
0.31515231373216374
0.3141653082776828
0.3159111596804887


In [79]:
datas = np.array([no6C, no7C, no8C, no9C, no10C, no11C])

In [80]:
def writeToCSV(data):
    l = len(data)
    for i, d in enumerate(data):
        with open('/gpfs/amarolab/monolayers/Monolayer_Sims_Raw_Data/MIX1_NaCl0.4_SA20_pH2/NematicOrder_'+str(i+6)+'.csv','w') as csv:
            for dat in d:
                csv.write(str(dat)+"\n")
writeToCSV(datas)